![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 7. Clinical NER Chunk Merger  v2.7.0

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.244.140:7077 
 AppName 
 Databricks Shell

In [0]:
# Sample data
data_chunk_merge = spark.createDataFrame([
  (1,"""A 63-year-old man presents to the hospital with a history of recurrent infections that include cellulitis, pneumonias, and upper respiratory tract infections. He reports subjective fevers at home along with unintentional weight loss and occasional night sweats. The patient has a remote history of arthritis, which was diagnosed approximately 20 years ago and treated intermittently with methotrexate (MTX) and prednisone. On physical exam, he is found to be febrile at 102°F, rather cachectic, pale, and have hepatosplenomegaly. Several swollen joints that are tender to palpation and have decreased range of motion are also present. His laboratory values show pancytopenia with the most severe deficiency in neutrophils.
""")]).toDF("id","text")

data_chunk_merge.show(truncate=50)

+---+--------------------------------------------------+
 id| text|
+---+--------------------------------------------------+
 1|A 63-year-old man presents to the hospital with...|
+---+--------------------------------------------------+

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = NerDLModel.pretrained("ner_deid_large", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("clinical_ner")

clinical_ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "clinical_ner"]) \
  .setOutputCol("clinical_ner_chunk")

# Cancer Genetics NER
bionlp_ner = NerDLModel.pretrained("ner_bionlp", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("bionlp_ner")

bionlp_ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "bionlp_ner"]) \
  .setOutputCol("bionlp_ner_chunk")

# merge ner_chunks by prioritizing the overlapping indices (chunks with longer lengths and highest information will be kept frome ach ner model)
chunk_merger_1 = ChunkMergeApproach()\
  .setInputCols('clinical_ner_chunk', "bionlp_ner_chunk")\
  .setOutputCol('clinical_bionlp_ner_chunk')

# internal clinical NER (general terms)
jsl_ner = NerDLModel.pretrained("ner_jsl", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("jsl_ner")

jsl_ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "jsl_ner"]) \
  .setOutputCol("jsl_ner_chunk")

# merge ner_chunks by prioritizing the overlapping indices (chunks with longer lengths and highest information will be kept frome ach ner model)
chunk_merger_2 = ChunkMergeApproach()\
  .setInputCols('clinical_bionlp_ner_chunk', "jsl_ner_chunk")\
  .setOutputCol('final_ner_chunk')

# merge ner_chunks regardess of overlapping indices 
# only works with 2.7 and later 
chunk_merger_NonOverlapped = ChunkMergeApproach()\
  .setInputCols('clinical_bionlp_ner_chunk', "jsl_ner_chunk")\
  .setOutputCol('nonOverlapped_ner_chunk')\
  .setMergeOverlapping(False)


nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    clinical_ner_converter,
    bionlp_ner,
    bionlp_ner_converter,
    chunk_merger_1,
    jsl_ner,
    jsl_ner_converter,
    chunk_merger_2,
    chunk_merger_NonOverlapped])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
ner_deid_large download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][OK!]
ner_jsl download started this may take some time.
Approximate size to download 14 MB
[ | ][ / ][ — ][OK!]

In [0]:
merged_data = model.transform(data_chunk_merge).cache()

In [0]:
from pyspark.sql import functions as F

result_df = merged_data.select('id',F.explode('final_ner_chunk').alias("cols")) \
.select('id',F.expr("cols.begin").alias("begin"),
        F.expr("cols.end").alias("end"),
        F.expr("cols.result").alias("chunk"),
        F.expr("cols.metadata.entity").alias("entity"))

result_df.show(50, truncate=100)


+---+-----+---+----------------------------------+----------------------+
 id|begin|end| chunk| entity|
+---+-----+---+----------------------------------+----------------------+
 1| 2| 12| 63-year-old| AGE|
 1| 14| 16| man| Organism|
 1| 61| 69| recurrent| Modifier|
 1| 95|104| cellulitis| Diagnosis|
 1| 107|116| pneumonias| Diagnosis|
 1| 123|156|upper respiratory tract infections| Diagnosis|
 1| 159|160| He| Gender|
 1| 170|179| subjective| Modifier|
 1| 181|186| fevers| Symptom_Name|
 1| 237|246| occasional| Modifier|
 1| 248|259| night sweats| Symptom_Name|
 1| 266|272| patient| Organism|
 1| 298|306| arthritis| Diagnosis|
 1| 343|344| 20| Age|
 1| 388|399| methotrexate| Simple_chemical|
 1| 402|405| MTX)| Drug_Name|
 1| 411|420| prednisone| Simple_chemical|
 1| 441|442| he| Gender|
 1| 459|465| febrile| Symptom_Name|
 1| 510|527| hepatosplenomegaly| Symptom_Name|
 1| 530|536| Several| Modifier|
 1| 538|551| swollen joints|Multi-tissue_structure|
 1| 635|637| His| Gender|
 1| 662|673| pancytopenia| Symptom_Name|
 1| 710|720| neutrophils| Cell|
+---+-----+---+----------------------------------+----------------------+

## NonOverlapped Chunk

all the entities form each ner model will be returned one by one

In [0]:
from pyspark.sql import functions as F

result_df2 = merged_data.select('id',F.explode('nonOverlapped_ner_chunk').alias("cols")) \
.select('id',F.expr("cols.begin").alias("begin"),
        F.expr("cols.end").alias("end"),
        F.expr("cols.result").alias("chunk"),
        F.expr("cols.metadata.entity").alias("entity"))

result_df2.show(50, truncate=100)


+---+-----+---+----------------------------------+----------------------+
 id|begin|end| chunk| entity|
+---+-----+---+----------------------------------+----------------------+
 1| 2| 12| 63-year-old| AGE|
 1| 2| 12| 63-year-old| Age|
 1| 14| 16| man| Organism|
 1| 14| 16| man| Gender|
 1| 61| 69| recurrent| Modifier|
 1| 95|104| cellulitis| Diagnosis|
 1| 107|116| pneumonias| Diagnosis|
 1| 123|156|upper respiratory tract infections| Diagnosis|
 1| 123|127| upper| Organism_subdivision|
 1| 141|145| tract| Organism_subdivision|
 1| 159|160| He| Gender|
 1| 170|179| subjective| Modifier|
 1| 181|186| fevers| Symptom_Name|
 1| 237|246| occasional| Modifier|
 1| 248|259| night sweats| Symptom_Name|
 1| 266|272| patient| Organism|
 1| 298|306| arthritis| Diagnosis|
 1| 343|344| 20| Age|
 1| 388|399| methotrexate| Simple_chemical|
 1| 388|399| methotrexate| Drug_Name|
 1| 402|405| MTX)| Drug_Name|
 1| 402|404| MTX| Simple_chemical|
 1| 411|420| prednisone| Simple_chemical|
 1| 411|420| prednisone| Drug_Name|
 1| 441|442| he| Gender|
 1| 459|465| febrile| Symptom_Name|
 1| 510|527| hepatosplenomegaly| Symptom_Name|
 1| 530|536| Several| Modifier|
 1| 538|551| swollen joints|Multi-tissue_structure|
 1| 538|551| swollen joints| Symptom_Name|
 1| 635|637| His| Gender|
 1| 662|673| pancytopenia| Symptom_Name|
 1| 710|720| neutrophils| Cell|
 1| 710|720| neutrophils| Lab_Name|
+---+-----+---+----------------------------------+----------------------+